# Generate Test Data

## test data for normal.mean

In [1]:
if(!require("devtools"))
{
    install.packages("devtools")      
}
library(devtools)

Loading required package: devtools

Loading required package: usethis



In [2]:
if(system.file(package='changed') != '')
{
    remove.packages("changed")
}
install_github("grosed/changed/R/package@dev")

Removing package from ‘/home/grosedj/work/changed-env/R-packages’
(as ‘lib’ is unspecified)





── R CMD build ─────────────────────────────────────────────────────────────────
✔  checking for file ‘/tmp/RtmpmArtOU/remotesfbbf0fc3ff2/grosed-changed-2110a6b/R/package/DESCRIPTION’
─  preparing ‘changed’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
     NB: this package now depends on R (>= 3.5.0)
     serialize/load version 3 cannot be read in older versions of R.
     File(s) containing such objects:
       ‘changed/tests/testthat/test.data.RData’
─  building ‘changed_1.0.tar.gz’
   


Installing package into ‘/home/grosedj/work/changed-env/R-packages’
(as ‘lib’ is unspecified)



In [3]:
library(changed)

## Generate Test Data

In [4]:
set.seed(0) 

singmean <- c(rnorm(100,0,1),rnorm(100,10,1))
mulmean <- c(rnorm(100,0,1),rnorm(100,10,1),rnorm(100,20,1),rnorm(100,50,1))
nochange <- c(rnorm(200,0,1))
singvar <- c(rnorm(100,10,1),rnorm(100,10,5))
mulvar <- c(rnorm(100,20,10),rnorm(100,20,15),rnorm(100,20,20),rnorm(100,20,25))
singmeanvar <- c(rnorm(50,0,1),rnorm(50,3,10))
mulmeanvar <- c(rnorm(50,0,1),rnorm(50,5,3),rnorm(50,10,1),rnorm(50,3,10))
mulmeanvarexp <- c(rexp(50,1), rexp(50,3), rexp(50,5), rexp(50,7)) 
mulmeanvarpois <- c(rpois(50,1), rpois(50,2), rpois(50,3), rpois(50,5)) 
constant <- rep(1, 200)

testdata <- list("singmean" = singmean,
                 "mulmean" = mulmean, 
                 "nochange" = nochange, 
                 "singvar" = singvar, 
                 "mulvar" = mulvar, 
                 "singmeanvar" = singmeanvar,
                 "mulmeanvar" = mulmeanvar, 
                 "mulmeanvarexp" = mulmeanvarexp, 
                 "mulmeanvarpois" = mulmeanvarpois,
                 "constant" = constant) 



## Test against changepoint 

In [5]:
if(system.file(package='changpoint') != '')
{
    remove.packages("changepoint")
}
install.packages("changepoint")

Installing package into ‘/home/grosedj/work/changed-env/R-packages’
(as ‘lib’ is unspecified)



In [6]:
library(changepoint)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Successfully loaded changepoint package version 2.2.4
 See NEWS for details of changes.



In [7]:
tests <- list()

In [8]:
normal_mean <- list()
number <- 1
for(name in names(testdata))
{
   n <- length(testdata[[name]])
   betas <- 2*log(n)*seq(0.0,2.0,0.05)
   for(beta in betas)
       {
          cpts <- cpt.mean(testdata[[name]],method="PELT",penalty="Manual",pen.value=beta)@cpts
          normal_mean[[number]] <- list(name,beta,cpts)
          number <- number + 1
       }
}


In [9]:
tests[["normal_mean"]] <- normal_mean

In [10]:
library(changed)
res <- c()
for(test in tests[["normal_mean"]])
{
    res <- c(res,setequal(test[[3]],changepoints(cpt.normal.mean(testdata[[test[[1]]]],test[[2]]))[,2]))
}

In [11]:
which(res == FALSE)

integer(0)

## Test against changepoint.np

In [12]:
if(system.file(package='changpoint.np') != '')
{
    remove.packages("changepoint.np")
}
install.packages("changepoint.np")

Installing package into ‘/home/grosedj/work/changed-env/R-packages’
(as ‘lib’ is unspecified)



In [13]:
library(changepoint.np)

In [14]:
np_average <- list()
number <- 1
for(name in names(testdata))
{
   n <- length(testdata[[name]])
   betas <- 4*log(n)*seq(0.0,2.0,0.05)
   ks <- seq(2,40,2)
   for(beta in betas)
       {
          for(k in ks)
              {
                  cpts <- cpt.np(testdata[[name]],method="PELT",penalty="Manual",pen.value=beta,nquantiles=k)@cpts
                  np_average[[number]] <- list(name,beta,k,cpts)
                  number <- number + 1
              }
       }
}

In [15]:
tests[["np_average"]] <- np_average

In [16]:
result <- c()
for(test in tests[["np_average"]])
{
    data <- testdata[[test[[1]]]]
    beta <- test[[2]]
    k <- test[[3]]
    qs <- quantiles(data,k,"non-equal")
    cpts <- test[[4]]
    result <- c(result,setequal(cpts,changepoints(cpt.np.average(data,qs,beta))[,2]))
}

In [17]:
which(result == FALSE)

[1]   23   24   25   26   27   30   31   32   33   34   35   39   40   42   43
 [16]   44   45   46   47   48   50   51   52   53   54   55   56   57   58   59
 [31]   60   61   62   64   65   66   67   68   69   70   72   73   74   75   76
 [46]   77   78   79   80   81   83   85   88   89   91   93   94   95   96   97
 [61]   98   99  100  101  102  104  105  107  108  109  110  116  117  118  119
 [76]  121  122  123  126  129  130  136  138  139  146  174  187  188  192  194
 [91]  195  197  843  844  845  846  848  850  851  852  853  854  857  858  859
[106]  860  862  863  865  868  869  870  871  872  873  875  877  878  879  881
[121]  882  883  884  886  889  891  892  894  897  899  900  901  909  913  914
[136]  921  922  925  941  943  946  949  951  955  956 1663 1664 1665 1666 1668
[151] 1670 1671 1672 1674 1677 1679 1680 1682 1683 1685 1687 1689 1690 1691 1693
[166] 1694 1696 1697 1701 1702 1706 1707 1708 1709 1710 1714 1716 1717 1718 1720
[181] 1721 1723 1726 1733 1735 1740 1741 1745 1746 1747 1748 1750 1751 1752 1753
[196] 1754 1755 1756 1757 1758 1759 1760 1761 1763 1770 1771 1776 1782 1788 1807
[211] 1808 1812 1817 1823 1825 1826 1829 1832 1834 1835 1836 1844 1847 1849 1851
[226] 1859 1864 2483 2484 2485 2486 2487 2490 2491 2492 2493 2494 2495 2497 2499
[241] 2502 2503 2504 2505 2507 2511 2512 2513 2514 2515 2516 2517 2520 2524 2525
[256] 2529 2541 2542 2544 2545 2546 2548 2550 2554 2557 2561 2562 2569 2570 2582
[271] 2590 2595 2596 2600 2603 2605 2607 2614 2621 2623 2627 2628 2630 2631 2634
[286] 2641 2643 2646 2649 2651 2652 2656 2657 2659 2660 2661 2672 2686 3303 3304
[301] 3305 3306 3309 3312 3313 3314 3317 3318 3319 3320 3322 3323 3324 3330 3332
[316] 3333 3334 3337 3342 3343 3344 3345 3346 3347 3349 3351 3352 3354 3355 3357
[331] 3359 3360 3361 3363 3365 3366 3371 3373 3376 3380 3381 3384 3387 3390 3391
[346] 3392 3396 3402 3405 3409 3411 3414 3417 3421 3422 3423 3441 3442 3461 3466
[361] 3474 3475 3480 3481 3487 3494 3501 4122 4123 4124 4125 4126 4127 4128 4129
[376] 4130 4131 4132 4133 4134 4135 4137 4138 4139 4140 4142 4143 4145 4146 4152
[391] 4154 4155 4159 4162 4163 4164 4166 4167 4168 4169 4170 4171 4172 4173 4174
[406] 4175 4176 4177 4178 4180 4181 4185 4188 4191 4201 4225 4244 4249 4261 4265
[421] 4272 4273 4281 4286 4290 4291 4296 4297 4298 4303 4306 4312 4314 4315 4316
[436] 4317 4319 4320 4943 4944 4945 4946 4947 4948 4950 4951 4952 4953 4954 4955
[451] 4956 4957 4958 4959 4960 4962 4963 4965 4966 4967 4968 4969 4970 4971 4973
[466] 4974 4975 4977 4978 4979 4981 4982 4984 4986 4988 4992 4994 4995 5001 5002
[481] 5006 5016 5017 5019 5021 5022 5041 5046 5048 5051 5054 5058 5061 5066 5071
[496] 5076 5084 5087 5089 5091 5092 5095 5098 5100 5111 5131 5151 5165 5166 5171
[511] 5191 5211 5231 5251 5271 5291 5311 5331 5351 5353 5366 5367 5376 5763 5764
[526] 5765 5766 5769 5770 5771 5772 5773 5778 5779 5780 5782 5783 5784 5787 5788
[541] 5790 5791 5793 5795 5796 5797 5799 5802 5804 5805 5811 5815 5816 5821 5823
[556] 5827 5828 5829 5830 5832 5835 5837 5839 5840 5841 5843 5849 5851 5852 5854
[571] 5856 5857 5859 5860 5861 5863 5864 5867 5881 5883 5884 5886 5888 5892 5893
[586] 5895 5897 5909 5910 5935 5946 5947 5955 5966 5986 5994 6005 6010 6017 6018
[601] 6020 6031 6561 6562 6563 6564 6565 6566 6567 6568 6569 6570 6571 6572 6573
[616] 6574 6575 6576 6577 6578 6579 6580 6581 6582 6583 6584 6585 6586 6587 6588
[631] 6589 6590 6591 6592 6593 6594 6595 6596 6597 6598 6599 6600 6601 6602 6603
[646] 6604 6605 6606 6607 6608 6609 6610 6611 6612 6613 6614 6615 6616 6617 6618
[661] 6619 6620 6621 6622 6623 6624 6625 6626 6627 6628 6629 6630 6631 6632 6633
[676] 6634 6635 6636 6637 6638 6639 6640 6641 6642 6643 6644 6645 6646 6647 6648
[691] 6649 6650 6651 6652 6653 6654 6655 6656 6657 6658 6659 6660 6661 6662 6663
[706] 6664 6665 6666 6667 6668 6669 6670 6671 6672 6673 6674 6675 6676 6677 6678
[721] 6679 6680 6681 6682 6683 6684 6685 6686 6687 6688 6689 6690 6691 6692 6693
[736] 6694 6695 6696 6697 6698 66

## Test generation of quantiles against Hyeyoung's code

### code for generating quantiles

In [18]:
### a sequence of quantiles
qnt.seq <- function(x, K=NULL, method="nonequally"){
  
  n <- length(x)
  if(is.null(K)){
    K <- round(4*log(n))
  }
  Q <- matrix(0, K, n+1)
  sorted.x = sort(x)
  yK = -1 + (2*(1:K)/K-1/K)
  c = -log(2*n-1)
  if(method=="equally"){
    pK = seq(0, 1, length.out = K+2)[-c(1, K+2)]
  }else{
    pK  = (1+exp(c*yK))^-1
  }
  
  qnt = rep(NA, K)
  for (i in 1:K){
    j  = as.integer((n-1)*pK[i] + 1)
    qnt[i] = sorted.x[j]
  }
  return(qnt)
  
}

In [19]:
quantile_values <- list()
number <- 1
for(name in names(testdata))
{
   n <- length(testdata[[name]])
   betas <- 2*log(n)*seq(0.0,2.0,0.05)
   Ks <- seq(2,20) 
   for(k in Ks)
       {
          qnts <- list(name,k,c(-Inf,qnt.seq(testdata[[name]],k),Inf))
          quantile_values[[number]] <- qnts
          number <- number + 1
       }
}

In [20]:
tests[["quantile_values"]] <- quantile_values

In [21]:
result <- c()
for(test in tests[["quantile_values"]])
{
    data <- testdata[[test[[1]]]]
    k <- test[[2]]
    qnts <- test[[3]]
    result <- c(result,setequal(qnts,quantiles(data,k,"non-equal")))
}

In [22]:
which(result == FALSE)

integer(0)

## Test Hyeyoung's average implementation aginst changepoint.np

In [23]:
### minus log-likelihood
minloglik <- function(x, Q, i, j){
  
  if (i>=j) {
    min.log.lik <- 0 
  }else{
    t <- j-i+1
    mi <- sum(x[i:j] < Q) + 0.5*sum(x[i:j]==Q)
    
    if(mi==0 | mi==t){
      min.log.lik <- 0 
    }else{
      min.log.lik <- -mi*log(mi/t)-(t-mi)*log(1-mi/t) 
    }
    
  }
  
  return(min.log.lik)  
  
}

In [24]:
### cost function for PELT
costMatrix = function(x, qnt) {
  
  N = length(x)
  K = length(qnt)
  C <- array(NA, dim=c(length(qnt), N, N))
  
  for(k in 1:K){
    for (i in 1:N){
      for (j in 1:N){
        C[k, i, j] = minloglik(x=x, Q=qnt[k], i=i, j=j)
      }
    }
  }
  
  return(C)
}

In [25]:
PELT = function (x, qnt, penalty) {
  
  # initialization
  K = length(qnt)
  
  C = costMatrix(x, qnt)
  
  N = length(x)
  
  
  Fvec = rep(0, N + 1)
  Fvec[1] = -penalty
  cngPoints = list(NULL) # initializing a null list
  Rvec = 0
  
  for (t in 1:N) {
    
    # compute all the costs up to time t
    meanC <- -2*log(2*N-1)*colMeans(-C[, (Rvec + 1), t, drop=F])
    partitionsCosts = Fvec[Rvec + 1] + meanC + penalty
    
    # get the new F(t) and its relative changepoint between R
    Fvec[(t) + 1] = min(partitionsCosts)
    cngPoint = Rvec[which.min(partitionsCosts)]
    cngPoints[[t+1]] = c(cngPoints[[cngPoint + 1]], cngPoint + 1)
    
    # make a vector of the same lenth to filter based on F(t)
    filter = (Fvec[Rvec + 1] + meanC) <= Fvec[(t) + 1]
    #print(c(filter))
    
    # append the new time to values in R that meet the condition
    Rvec = c(Rvec[filter], t)
    
  }
  
  return(cngPoints[[N]][-1])
  
}

In [33]:
test <- tests[["np_average"]][[1000]]

In [34]:
data <- testdata[[test[[1]]]]
beta <- test[[2]]
k <- test[[3]]
qs <- qnt.seq(data,k)
cpts <- test[[4]]
print(k)
print(qs)
print(cpts)

[1] 40
 [1] -2.9048991 -2.9048991 -1.8900271 -1.8900271 -1.8083276 -1.7145023
 [7] -1.6482217 -1.5967180 -1.4755476 -1.1985971 -1.0662002 -0.8874201
[13] -0.3654018 -0.1472708  0.1054214  0.6169665  1.6969479  9.1654410
[19] 10.0389281 10.8373944 18.8213276 19.8545092 20.8037575 48.2601649
[25] 49.3736318 49.7124175 50.0187845 50.3973491 50.6441970 50.8335890
[31] 51.1013041 51.3187011 51.4742449 51.5192970 51.5497130 51.6952711
[37] 51.6998624 51.6998624 51.8499196 51.8499196
[1] 100 200 300 318 396 400


In [35]:
setequal(cpts,c(PELT(data,qs,beta)-1,length(data)))

[1] TRUE

## Test cpt.np.conditional and cpt.np.max Hyeyoung's average implementations

In [26]:
costMatrix_c = function(x, qnt) {
  
  N = length(x)
  K = length(qnt)
  
  C = matrix(nr = N, nc = N)
  qnt0 = c(0, qnt, Inf)
  
  for (i in 1:N){
    
    for (j in 1:N){
      
      if (i>=j) {
        
        min.log.lik <- 0 
        
      }else{
        
        t <- j-i+1
        mi <- rep(NA, (length(qnt0)-1))
        f_mi <- rep(NA, (length(qnt0)-1))
        
        for(l in 1:(length(qnt0)-1)){
          
          mi[l] <- sum(((qnt0[l] < x[i:j])+(x[i:j] <= qnt0[l+1]))==2)
          
          if(mi[l]==0 | mi[l]==t){
            f_mi[l] <- 0 
          }else{
            f_mi[l] <- -mi[l]*log(mi[l]/t)
          }
          
        }
        
        min.log.lik <- sum(f_mi)
        
      }
      
      C[i, j] = min.log.lik
      
    }
  }
  
  return(C)
}

In [27]:
### PELT-conditional (section 2.2)
PELT_c = function (x, qnt, penalty) {
  
  # initialization
  K = length(qnt)
  
  C = costMatrix_c(x, qnt)
  
  N = length(x)
  
  # penalty = K + 2*sqrt(K*thr.c*log(N)) + 2*thr.c*log(N)
  
  Fvec = rep(0, N + 1)
  Fvec[1] = -penalty
  cngPoints = list(NULL) # initializing a null list
  Rvec = 0
  
  for (t in 1:N) {
    
    # compute all the costs up to time t
    aggregatedC <- C[(Rvec + 1), t, drop=F]

    partitionsCosts = Fvec[Rvec + 1] + aggregatedC + penalty
    
    # get the new F(t) and its relative changepoint between R
    Fvec[(t) + 1] = min(partitionsCosts)
    cngPoint = Rvec[which.min(partitionsCosts)]
    cngPoints[[t+1]] = c(cngPoints[[cngPoint + 1]], cngPoint + 1)
    
    # make a vector of the same lenth to filter based on F(t)
    filter = (Fvec[Rvec + 1] + aggregatedC) <= Fvec[(t) + 1]
    
    # append the new time to values in R that meet the condition
    Rvec = c(Rvec[filter], t)
    
  }
  
  return(cngPoints[[N]][-1])
  
}

In [28]:
PELT_max = function (x, qnt, penalty) {
  
  # initialization
  K = length(qnt)
  
  C = costMatrix(x, qnt)
  
  N = length(x)
  
  # penalty = thr.c*log(N)
  #penalty = 1 + 2*sqrt(1*thr.c*log(N)) + 2*thr.c*log(N)
  
  Fvec = rep(0, N + 1)
  Fvec[1] = -penalty
  cngPoints = list(NULL) # initializing a null list
  Rvec = 0
  
  for (t in 1:N) {
    
    # compute all the costs up to time t
    maxC <- apply(2*C[, (Rvec + 1), t, drop=F], 2, max)
    partitionsCosts = Fvec[Rvec + 1] + maxC + penalty
    
    # get the new F(t) and its relative changepoint between R
    Fvec[(t) + 1] = min(partitionsCosts)
    cngPoint = Rvec[which.min(partitionsCosts)]
    cngPoints[[t+1]] = c(cngPoints[[cngPoint + 1]], cngPoint + 1)
    
    # make a vector of the same lenth to filter based on F(t)
    filter = (Fvec[Rvec + 1] + maxC) <= Fvec[(t) + 1]
    #print(c(filter))
    
    # append the new time to values in R that meet the condition
    Rvec = c(Rvec[filter], t)
    
  }
  
  return(cngPoints[[N]][-1])
  
}

In [82]:
test <- tests[["np_average"]][[1005]]
data <- testdata[[test[[1]]]]
#beta <- test[[2]]
beta <- 1*log(length(data))
k <- test[[3]]
# k <- 40
qs <- qnt.seq(data,k)
cpts <- test[[4]]
print(beta)
print(k)
print(qs)
print(cpts)


[1] 5.991465
[1] 10
 [1] -2.904899060 -1.714502297 -1.113122953  0.005641985  9.698803017
 [6] 20.459621673 49.805153616 50.965761296 51.549712963 51.849919560
[1] 100 202 301 318 400


In [53]:
print("cpt.np.average")
print(changepoints(cpt.np.average(data,c(-Inf,qs,Inf),beta)))
print("cpt.np")
cpt.np(data,method="PELT",penalty="Manual",pen.value=beta,nquantiles=k)@cpts
print("hyeyoung average :")
print(c(PELT(data,qs,beta)-1,length(data)))
print("cpt.np.conditional")
print(changepoints(cpt.np.conditional(data,c(-Inf,qs,Inf),beta)))
print("hyeyoung conditional :")
print(c(PELT_c(data,qs,beta)-1,length(data)))
print("cpt.np.max")
print(changepoints(cpt.np.max(data,c(-Inf,qs,Inf),beta)))
print(changepoints(cpt.np.max(data,c(-Inf,qs),beta)))
print(changepoints(cpt.np.max(data,c(qs,Inf),beta)))
print(changepoints(cpt.np.max(data,qs,beta)))
print("hyeyoung max :")
print(c(PELT_max(data,qs,beta)-1,length(data)))

[1] "cpt.np.average"
   start end      cost
1      1  44 63.208820
2     45  45  0.000000
3     46  66 28.941805
4     67  69  6.065019
5     70 100 50.445650
6    101 200 55.608232
7    201 300 55.507728
8    301 307 13.815467
9    308 308  0.000000
10   309 317 11.075902
11   318 318  0.000000
12   319 356 61.587973
13   357 364  6.152877
14   365 396 29.566263
15   397 400  8.736838
[1] "cpt.np"


[1]  44  45  66  69 100 200 300 307 308 317 318 356 364 396 400

[1] "hyeyoung average :"
 [1]  44  45  66  69 100 200 300 307 308 317 318 356 364 396 400
[1] "cpt.np.conditional"
   start end       cost
1      1   2   0.000000
2      3  25  39.726254
3     26  32  10.848782
4     33  43  17.012586
5     44  65  40.887421
6     66  74  15.092890
7     75  85  16.149539
8     86  94  15.092890
9     95 100   6.068426
10   101 200 135.477950
11   201 300 135.477950
12   301 304   5.545177
13   305 309   4.751353
14   310 316   8.939240
15   317 330  24.470153
16   331 335   3.365058
17   336 348  21.207490
18   349 354   7.977968
19   355 371  30.413377
20   372 394  34.319519
21   395 400   9.364262
[1] "hyeyoung conditional :"
 [1]   2  25  32  43  65  74  85  94 100 200 300 304 309 316 330 335 348 354 360
[20] 364 380 393 400
[1] "cpt.np.max"
  start end     cost
1     1 400 551.7921
  start end     cost
1     1 400 551.7921
  start end     cost
1     1 400 551.7921
  start end     cost
1     1 400 551.7921
[1] "hyeyoung max :"
[1] 400


In [83]:
cf <-new(changed::np_conditional)
cf$set(data,c(-Inf,qs,Inf))
cm <- costMatrix_c(data,qs)


In [84]:
i <- 50
j <- length(data)-100
print(cf$cost(i,j))
print(cm[i,j])

[1] 316.4396
[1] 316.4396


In [85]:
comp <- matrix(TRUE,length(data),length(data))
for(i in 1:length(data))
    {
        for(j in 1:length(data))
        {
            comp[i,j] <- all.equal(cf$cost(i,j),cm[i,j])
            #comp <- c(comp,abs((cf$cost(i,j) - cfh(cm,i,j,length(data)))/cf$cost(i,j)))
        }
    }

In [87]:
which(comp != TRUE)

[1]  17601  17602  17603  17604  17605  17606  17607  17608  17609  17610
   [11]  17611  17612  17613  17614  17615  17616  17617  17618  17619  17620
   [21]  17621  17622  17623  17624  17625  17626  17627  17628  17629  17630
   [31]  17631  17632  17633  17634  17635  17636  17637  17638  17639  17640
   [41]  17641  17642  17643  17644  18001  18002  18003  18004  18005  18006
   [51]  18007  18008  18009  18010  18011  18012  18013  18014  18015  18016
   [61]  18017  18018  18019  18020  18021  18022  18023  18024  18025  18026
   [71]  18027  18028  18029  18030  18031  18032  18033  18034  18035  18036
   [81]  18037  18038  18039  18040  18041  18042  18043  18044  18045  18401
   [91]  18402  18403  18404  18405  18406  18407  18408  18409  18410  18411
  [101]  18412  18413  18414  18415  18416  18417  18418  18419  18420  18421
  [111]  18422  18423  18424  18425  18426  18427  18428  18429  18430  18431
  [121]  18432  18433  18434  18435  18436  18437  18438  18439  18440  18441
  [131]  18442  18443  18444  18445  18801  18802  18803  18804  18805  18806
  [141]  18807  18808  18809  18810  18811  18812  18813  18814  18815  18816
  [151]  18817  18818  18819  18820  18821  18822  18823  18824  18825  18826
  [161]  18827  18828  18829  18830  18831  18832  18833  18834  18835  18836
  [171]  18837  18838  18839  18840  18841  18842  18843  18844  18845  19201
  [181]  19202  19203  19204  19205  19206  19207  19208  19209  19210  19211
  [191]  19212  19213  19214  19215  19216  19217  19218  19219  19220  19221
  [201]  19222  19223  19224  19225  19226  19227  19228  19229  19230  19231
  [211]  19232  19233  19234  19235  19236  19237  19238  19239  19240  19241
  [221]  19242  19243  19244  19245  19601  19602  19603  19604  19605  19606
  [231]  19607  19608  19609  19610  19611  19612  19613  19614  19615  19616
  [241]  19617  19618  19619  19620  19621  19622  19623  19624  19625  19626
  [251]  19627  19628  19629  19630  19631  19632  19633  19634  19635  19636
  [261]  19637  19638  19639  19640  19641  19642  19643  19644  19645  20001
  [271]  20002  20003  20004  20005  20006  20007  20008  20009  20010  20011
  [281]  20012  20013  20014  20015  20016  20017  20018  20019  20020  20021
  [291]  20022  20023  20024  20025  20026  20027  20028  20029  20030  20031
  [301]  20032  20033  20034  20035  20036  20037  20038  20039  20040  20041
  [311]  20042  20043  20044  20045  20401  20402  20403  20404  20405  20406
  [321]  20407  20408  20409  20410  20411  20412  20413  20414  20415  20416
  [331]  20417  20418  20419  20420  20421  20422  20423  20424  20425  20426
  [341]  20427  20428  20429  20430  20431  20432  20433  20434  20435  20436
  [351]  20437  20438  20439  20440  20441  20442  20443  20444  20445  20801
  [361]  20802  20803  20804  20805  20806  20807  20808  20809  20810  20811
  [371]  20812  20813  20814  20815  20816  20817  20818  20819  20820  20821
  [381]  20822  20823  20824  20825  20826  20827  20828  20829  20830  20831
  [391]  20832  20833  20834  20835  20836  20837  20838  20839  20840  20841
  [401]  20842  20843  20844  20845  21201  21202  21203  21204  21205  21206
  [411]  21207  21208  21209  21210  21211  21212  21213  21214  21215  21216
  [421]  21217  21218  21219  21220  21221  21222  21223  21224  21225  21226
  [431]  21227  21228  21229  21230  21231  21232  21233  21234  21235  21236
  [441]  21237  21238  21239  21240  21241  21242  21243  21244  21245  21601
  [451]  21602  21603  21604  21605  21606  21607  21608  21609  21610  21611
  [461]  21612  21613  21614  21615  21616  21617  21618  21619  21620  21621
  [471]  21622  21623  21624  21625  21626  21627  21628  21629  21630  21631
  [481]  21632  21633  21634  21635  21636  21637  21638  21639  21640  21641
  [491]  21642  21643  21644  21645  22001  22002  22003  22004  22005  22006
  [501]  22007  22008  22009  22010  22011  22012  22013  22014  22015  22016
  [511]  22017  22018  220

In [70]:
which(comp != TRUE)

[1]      1    401    801   1201   1601   2001   2401   2801   3201   3601
   [11]   4001   4401   4801   5201   5601   6001   6401   6801   7201   7601
   [21]   8001   8401   8801   9201   9601  10001  10401  10801  11201  11601
   [31]  12001  12401  12801  13201  13601  14001  14401  14801  15201  15601
   [41]  16001  16401  16801  17201  17601  17602  17603  17604  17605  17606
   [51]  17607  17608  17609  17610  17611  17612  17613  17614  17615  17616
   [61]  17617  17618  17619  17620  17621  17622  17623  17624  17625  17626
   [71]  17627  17628  17629  17630  17631  17632  17633  17634  17635  17636
   [81]  17637  17638  17639  17640  17641  17642  17643  17644  18001  18002
   [91]  18003  18004  18005  18006  18007  18008  18009  18010  18011  18012
  [101]  18013  18014  18015  18016  18017  18018  18019  18020  18021  18022
  [111]  18023  18024  18025  18026  18027  18028  18029  18030  18031  18032
  [121]  18033  18034  18035  18036  18037  18038  18039  18040  18041  18042
  [131]  18043  18044  18045  18401  18402  18403  18404  18405  18406  18407
  [141]  18408  18409  18410  18411  18412  18413  18414  18415  18416  18417
  [151]  18418  18419  18420  18421  18422  18423  18424  18425  18426  18427
  [161]  18428  18429  18430  18431  18432  18433  18434  18435  18436  18437
  [171]  18438  18439  18440  18441  18442  18443  18444  18445  18801  18802
  [181]  18803  18804  18805  18806  18807  18808  18809  18810  18811  18812
  [191]  18813  18814  18815  18816  18817  18818  18819  18820  18821  18822
  [201]  18823  18824  18825  18826  18827  18828  18829  18830  18831  18832
  [211]  18833  18834  18835  18836  18837  18838  18839  18840  18841  18842
  [221]  18843  18844  18845  19201  19202  19203  19204  19205  19206  19207
  [231]  19208  19209  19210  19211  19212  19213  19214  19215  19216  19217
  [241]  19218  19219  19220  19221  19222  19223  19224  19225  19226  19227
  [251]  19228  19229  19230  19231  19232  19233  19234  19235  19236  19237
  [261]  19238  19239  19240  19241  19242  19243  19244  19245  19601  19602
  [271]  19603  19604  19605  19606  19607  19608  19609  19610  19611  19612
  [281]  19613  19614  19615  19616  19617  19618  19619  19620  19621  19622
  [291]  19623  19624  19625  19626  19627  19628  19629  19630  19631  19632
  [301]  19633  19634  19635  19636  19637  19638  19639  19640  19641  19642
  [311]  19643  19644  19645  20001  20002  20003  20004  20005  20006  20007
  [321]  20008  20009  20010  20011  20012  20013  20014  20015  20016  20017
  [331]  20018  20019  20020  20021  20022  20023  20024  20025  20026  20027
  [341]  20028  20029  20030  20031  20032  20033  20034  20035  20036  20037
  [351]  20038  20039  20040  20041  20042  20043  20044  20045  20401  20402
  [361]  20403  20404  20405  20406  20407  20408  20409  20410  20411  20412
  [371]  20413  20414  20415  20416  20417  20418  20419  20420  20421  20422
  [381]  20423  20424  20425  20426  20427  20428  20429  20430  20431  20432
  [391]  20433  20434  20435  20436  20437  20438  20439  20440  20441  20442
  [401]  20443  20444  20445  20801  20802  20803  20804  20805  20806  20807
  [411]  20808  20809  20810  20811  20812  20813  20814  20815  20816  20817
  [421]  20818  20819  20820  20821  20822  20823  20824  20825  20826  20827
  [431]  20828  20829  20830  20831  20832  20833  20834  20835  20836  20837
  [441]  20838  20839  20840  20841  20842  20843  20844  20845  21201  21202
  [451]  21203  21204  21205  21206  21207  21208  21209  21210  21211  21212
  [461]  21213  21214  21215  21216  21217  21218  21219  21220  21221  21222
  [471]  21223  21224  21225  21226  21227  21228  21229  21230  21231  21232
  [481]  21233  21234  21235  21236  21237  21238  21239  21240  21241  21242
  [491]  21243  21244  21245  21601  21602  21603  21604  21605  21606  21607
  [501]  21608  21609  21610  21611  21612  21613  21614  21615  21616  21617
  [511]  21618  21619  216

In [73]:
comp[400]

[1] "TRUE"

In [164]:
cf$cost(4,1)

[1] -1.386294

In [169]:
which(cm == 0)


[1]      1      2      3      4      5      6      7      8      9     10
   [11]     11     12     13     14     15     16     17     18     19     20
   [21]     21     22     23     24     25     26     27     28     29     30
   [31]     31     32     33     34     35     36     37     38     39     40
   [41]     41     42     43     44     45     46     47     48     49     50
   [51]     51     52     53     54     55     56     57     58     59     60
   [61]     61     62     63     64     65     66     67     68     69     70
   [71]     71     72     73     74     75     76     77     78     79     80
   [81]     81     82     83     84     85     86     87     88     89     90
   [91]     91     92     93     94     95     96     97     98     99    100
  [101]    101    102    103    104    105    106    107    108    109    110
  [111]    111    112    113    114    115    116    117    118    119    120
  [121]    121    122    123    124    125    126    127    128    129    130
  [131]    131    132    133    134    135    136    137    138    139    140
  [141]    141    142    143    144    145    146    147    148    149    150
  [151]    151    152    153    154    155    156    157    158    159    160
  [161]    161    162    163    164    165    166    167    168    169    170
  [171]    171    172    173    174    175    176    177    178    179    180
  [181]    181    182    183    184    185    186    187    188    189    190
  [191]    191    192    193    194    195    196    197    198    199    200
  [201]    201    202    203    204    205    206    207    208    209    210
  [211]    211    212    213    214    215    216    217    218    219    220
  [221]    221    222    223    224    225    226    227    228    229    230
  [231]    231    232    233    234    235    236    237    238    239    240
  [241]    241    242    243    244    245    246    247    248    249    250
  [251]    251    252    253    254    255    256    257    258    259    260
  [261]    261    262    263    264    265    266    267    268    269    270
  [271]    271    272    273    274    275    276    277    278    279    280
  [281]    281    282    283    284    285    286    287    288    289    290
  [291]    291    292    293    294    295    296    297    298    299    300
  [301]    301    302    303    304    305    306    307    308    309    310
  [311]    311    312    313    314    315    316    317    318    319    320
  [321]    321    322    323    324    325    326    327    328    329    330
  [331]    331    332    333    334    335    336    337    338    339    340
  [341]    341    342    343    344    345    346    347    348    349    350
  [351]    351    352    353    354    355    356    357    358    359    360
  [361]    361    362    363    364    365    366    367    368    369    370
  [371]    371    372    373    374    375    376    377    378    379    380
  [381]    381    382    383    384    385    386    387    388    389    390
  [391]    391    392    393    394    395    396    397    398    399    400
  [401]    401    402    403    404    405    406    407    408    409    410
  [411]    411    412    413    414    415    416    417    418    419    420
  [421]    421    422    423    424    425    426    427    428    429    430
  [431]    431    432    433    434    435    436    437    438    439    440
  [441]    441    442    443    444    445    446    447    448    449    450
  [451]    451    452    453    454    455    456    457    458    459    460
  [461]    461    462    463    464    465    466    467    468    469    470
  [471]    471    472    473    474    475    476    477    478    479    480
  [481]    481    482    483    484    485    486    487    488    489    490
  [491]    491    492    493    494    495    496    497    498    499    500
  [501]    501    502    503    504    505    506    507    508    509    510
  [511]    511    512    5

In [35]:
cf <-new(changed::np_average)
cf$set(data,c(-Inf,qs,Inf))
cm <- costMatrix(data,qs)
n <- length(data)
cf_average <- function(C,i,j,N)
    {
      return((-2*log(2*N-1)*colMeans(-C[, i, j, drop=F]))[1,1])
    }
cfh <- cf_average

In [36]:
i <- 50
j <- length(data)-100
print(cf$cost(i,j))
print(cfh(cm,i,j,length(data)))

[1] 520.0696
[1] 520.0696


In [37]:
comp <- matrix(0,length(data),length(data))
for(i in 1:length(data))
    {
        for(j in 1:length(data))
        {
            comp[i,j] <- all.equal(cf$cost(i,j),cfh(cm,i,j,length(data)))
            #comp <- c(comp,abs((cf$cost(i,j) - cfh(cm,i,j,length(data)))/cf$cost(i,j)))
        }
    }

In [48]:
which(comp != 1)

integer(0)

In [50]:
cf <-new(changed::np_max)
cf$set(data,c(-Inf,qs,Inf))
cm <- costMatrix(data,qs)
n <- length(data)
cf_max <- function(C,i,j)
    {
    return(apply(2*C[, i, j, drop=F], 2, max))
    }
cfh <- cf_max

In [51]:
comp <- matrix(0,length(data),length(data))
for(i in 1:length(data))
    {
        for(j in 1:length(data))
        {
            comp[i,j] <- all.equal(cf$cost(i,j),cfh(cm,i,j))
            #comp <- c(comp,abs((cf$cost(i,j) - cfh(cm,i,j,length(data)))/cf$cost(i,j)))
        }
    }

In [47]:
which(comp != 1)

integer(0)

In [30]:
a <- 7
b <- 7
all.equal(cf$cost(a,b),cfh(cm,a,b,length(data)))

[1] "Mean relative difference: 1"

In [31]:
cf$cost(a,b)


[1] 7.907108

In [32]:
cfh(cm,a,b,length(data))

[1] 0

In [98]:
print(cf$cost(1,4))
print(cfh(cm,1,4,length(data)))

[1] 4.4577
[1] 4.4577


In [99]:
cf$cost(1,4)

[1] 4.4577

In [101]:
cfh(cm,1,4,length(data))

[1] 4.4577

In [104]:
all.equal(cf$cost(1,4),cfh(cm,1,4,length(data)))

[1] TRUE

In [41]:
library(changed)

In [42]:
np <- new(np_average)

ERROR: Error in .getClassesFromCache(Class): class should be either a character-string name or a class definition


In [64]:
changepoints(cpt.np.conditional(data,qs,beta))

start,end,cost
<dbl>,<dbl>,<dbl>
1,65,135.54648
66,73,11.95340
74,100,54.58971
101,200,135.47795
201,300,135.47795
301,316,32.22457
317,340,44.87167
341,400,125.23234


[1] 400